In [1]:
print ("============================================================")
print (" Course Work                              Robiul Islam      ")
print ("============================================================")


import pandas as pd 
from sklearn.utils import shuffle
from itertools import combinations
import bisect
import math

data = pd.read_csv("datafull.csv",sep=';') 
gen = pd.read_csv('/Users/HSE/coursework/python_vmv/SeqDem/gen.csv')
data['Genaration']=gen
# Remove Null value
data.dropna(inplace = True)
data = shuffle(data, random_state=7)
data.head()
def power_set(List):
    PS = [list(j) for i in range(len(List)) for j in combinations(List, i+1)]
    return PS

def shapley_calc(N,characteristic_function,n):
    shapley_values = []
    for i in range(n):
        shapley = 0
        for j in N:
            if i not in j:
                try:
                    cmod = len(j)
                    Cui = j[:]
                    l = N.index(j)
                    k = N.index(Cui)
                    temp = float(float(characteristic_function[k]) - float(characteristic_function[l])) *\
                               float(math.factorial(cmod) * math.factorial(n - cmod - 1)) / float(math.factorial(n))
                    shapley += temp
                except:
                    temp = float(N[0][0])
                    shapley += temp

        shapley_values.append(shapley)
    return shapley_values 

result =[]
for row in range(data.shape[0]):
    PS = power_set(data.iloc[row])
    Chan_fun =[]
    for i in PS:
        Chan_fun.append(max(i))
    Value = shapley_calc(PS,Chan_fun,5)
    result.append(max(Value))
    
data['result'] = result
print(data.head())

# gen = pd.read_csv('/Users/HSE/coursework/python_vmv/SeqDem/gen.csv')
# print(gen.head())

 Course Work                              Robiul Islam      
      work  separation  partner  marriage  children  Genaration  result
283   18.0        20.0     20.0      21.0      22.0           6   126.0
1329  17.0        25.0     20.0      20.0      29.0           4   119.0
2051  25.0        25.0     25.0      34.0      35.0           5   175.0
450   17.0        19.0     30.0      24.0      26.0           4   119.0
1135  19.0        19.0     23.0      24.0      25.0           2   133.0


In [14]:

from keras.models import Sequential
from keras.layers import Dense
import numpy
# fix random seed for reproducibility
numpy.random.seed(7)
batch_size = 5
# load pima indians dataset
# split into input (X) and output (Y) variables
X = data[['work','separation','partner','marriage','children','result']].values
Y = data['Genaration'].values
# create model

score =[]
for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = Sequential()
    model.add(Dense(12, input_dim=6, activation='relu'))
    model.add(Dense(6, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
# Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# Fit the model
    model.fit(X_train, y_train, epochs=100,batch_size=batch_size)
# evaluate the model
    s = model.evaluate(X_test, y_test)
    score.append(s)
    print("\n%s: %.2f%%" % (model.metrics_names[1], s[1]*100))

Epoch 1/100
247/247 [==============================] - 1s 3ms/step - loss: -47.9477 - acc: 0.0202
Epoch 2/100
247/247 [==============================] - 0s 337us/step - loss: -49.3089 - acc: 0.0202
Epoch 3/100
247/247 [==============================] - 0s 340us/step - loss: -49.3089 - acc: 0.0202
Epoch 4/100
247/247 [==============================] - 0s 339us/step - loss: -49.3117 - acc: 0.0202
Epoch 5/100
247/247 [==============================] - 0s 371us/step - loss: -49.3117 - acc: 0.0202
Epoch 6/100
247/247 [==============================] - 0s 387us/step - loss: -49.3117 - acc: 0.0202
Epoch 7/100
247/247 [==============================] - 0s 370us/step - loss: -49.3117 - acc: 0.0202
Epoch 8/100
247/247 [==============================] - 0s 354us/step - loss: -49.3117 - acc: 0.0202
Epoch 9/100
247/247 [==============================] - 0s 359us/step - loss: -49.3117 - acc: 0.0202
Epoch 10/100
247/247 [==============================] - 0s 351us/step - loss: -49.3117 - acc: 0.0202
E

247/247 [==============================] - 0s 370us/step - loss: -49.3117 - acc: 0.0202
Epoch 83/100
247/247 [==============================] - 0s 326us/step - loss: -49.3117 - acc: 0.0202
Epoch 84/100
247/247 [==============================] - 0s 307us/step - loss: -49.3117 - acc: 0.0202
Epoch 85/100
247/247 [==============================] - 0s 312us/step - loss: -49.3117 - acc: 0.0202
Epoch 86/100
247/247 [==============================] - 0s 463us/step - loss: -49.3117 - acc: 0.0202
Epoch 87/100
247/247 [==============================] - 0s 1ms/step - loss: -49.3117 - acc: 0.0202
Epoch 88/100
247/247 [==============================] - 0s 963us/step - loss: -49.3117 - acc: 0.0202
Epoch 89/100
247/247 [==============================] - 0s 488us/step - loss: -49.3117 - acc: 0.0202
Epoch 90/100
247/247 [==============================] - 0s 404us/step - loss: -49.3117 - acc: 0.0202
Epoch 91/100
247/247 [==============================] - 0s 330us/step - loss: -49.3117 - acc: 0.0202
Epoch

247/247 [==============================] - 0s 646us/step - loss: -49.3117 - acc: 0.0202
Epoch 63/100
247/247 [==============================] - 0s 364us/step - loss: -49.3117 - acc: 0.0202
Epoch 64/100
247/247 [==============================] - 0s 313us/step - loss: -49.3117 - acc: 0.0202
Epoch 65/100
247/247 [==============================] - 0s 327us/step - loss: -49.3117 - acc: 0.0202
Epoch 66/100
247/247 [==============================] - 0s 321us/step - loss: -49.3117 - acc: 0.0202
Epoch 67/100
247/247 [==============================] - 0s 322us/step - loss: -49.3117 - acc: 0.0202
Epoch 68/100
247/247 [==============================] - 0s 335us/step - loss: -49.3117 - acc: 0.0202
Epoch 69/100
247/247 [==============================] - 0s 347us/step - loss: -49.3117 - acc: 0.0202
Epoch 70/100
247/247 [==============================] - 0s 402us/step - loss: -49.3117 - acc: 0.0202
Epoch 71/100
247/247 [==============================] - 0s 684us/step - loss: -49.3117 - acc: 0.0202ETA:

247/247 [==============================] - 0s 369us/step - loss: -5.9310 - acc: 0.0202
Epoch 43/100
247/247 [==============================] - 0s 337us/step - loss: -6.0827 - acc: 0.0202
Epoch 44/100
247/247 [==============================] - 0s 653us/step - loss: -6.2333 - acc: 0.0202
Epoch 45/100
247/247 [==============================] - 0s 355us/step - loss: -6.3836 - acc: 0.0202
Epoch 46/100
247/247 [==============================] - 0s 351us/step - loss: -6.5346 - acc: 0.0202
Epoch 47/100
247/247 [==============================] - 0s 336us/step - loss: -6.6856 - acc: 0.0202
Epoch 48/100
247/247 [==============================] - 0s 328us/step - loss: -6.8358 - acc: 0.0202
Epoch 49/100
247/247 [==============================] - 0s 352us/step - loss: -6.9872 - acc: 0.0202
Epoch 50/100
247/247 [==============================] - 0s 352us/step - loss: -7.1382 - acc: 0.0202
Epoch 51/100
247/247 [==============================] - 0s 327us/step - loss: -7.2886 - acc: 0.0202
Epoch 52/100


Epoch 23/100
247/247 [==============================] - 0s 345us/step - loss: -49.3117 - acc: 0.0202
Epoch 24/100
247/247 [==============================] - 0s 325us/step - loss: -49.3117 - acc: 0.0202
Epoch 25/100
247/247 [==============================] - 0s 327us/step - loss: -49.3117 - acc: 0.0202
Epoch 26/100
247/247 [==============================] - 0s 341us/step - loss: -49.3117 - acc: 0.0202
Epoch 27/100
247/247 [==============================] - 0s 367us/step - loss: -49.3117 - acc: 0.0202
Epoch 28/100
247/247 [==============================] - 0s 376us/step - loss: -49.3117 - acc: 0.0202
Epoch 29/100
247/247 [==============================] - 0s 350us/step - loss: -49.3117 - acc: 0.0202
Epoch 30/100
247/247 [==============================] - 0s 378us/step - loss: -49.3117 - acc: 0.0202
Epoch 31/100
247/247 [==============================] - 0s 400us/step - loss: -49.3117 - acc: 0.0202
Epoch 32/100
247/247 [==============================] - 0s 359us/step - loss: -49.3117 - ac

...

247/247 [==============================] - 0s 389us/step - loss: -49.3117 - acc: 0.0202
Epoch 45/100
247/247 [==============================] - 0s 341us/step - loss: -49.3117 - acc: 0.0202
Epoch 46/100
247/247 [==============================] - 0s 335us/step - loss: -49.3117 - acc: 0.0202
Epoch 47/100
247/247 [==============================] - 0s 334us/step - loss: -49.3117 - acc: 0.0202
Epoch 48/100
247/247 [==============================] - 0s 359us/step - loss: -49.3117 - acc: 0.0202
Epoch 49/100
247/247 [==============================] - 0s 366us/step - loss: -49.3117 - acc: 0.0202
Epoch 50/100
247/247 [==============================] - 0s 323us/step - loss: -49.3117 - acc: 0.0202
Epoch 51/100
247/247 [==============================] - 0s 327us/step - loss: -49.3117 - acc: 0.0202
Epoch 52/100
247/247 [==============================] - 0s 331us/step - loss: -49.3117 - acc: 0.0202
Epoch 53/100
247/247 [==============================] - 0s 347us/step - loss: -49.3117 - acc: 0.0202
Epo

247/247 [==============================] - 0s 396us/step - loss: -49.3117 - acc: 0.0202
Epoch 25/100
247/247 [==============================] - 0s 354us/step - loss: -49.3117 - acc: 0.0202
Epoch 26/100
247/247 [==============================] - 0s 344us/step - loss: -49.3117 - acc: 0.0202
Epoch 27/100
247/247 [==============================] - 0s 345us/step - loss: -49.3117 - acc: 0.0202
Epoch 28/100
247/247 [==============================] - 0s 341us/step - loss: -49.3117 - acc: 0.0202
Epoch 29/100
247/247 [==============================] - 0s 340us/step - loss: -49.3117 - acc: 0.0202
Epoch 30/100
247/247 [==============================] - 0s 342us/step - loss: -49.3117 - acc: 0.0202
Epoch 31/100
247/247 [==============================] - 0s 344us/step - loss: -49.3117 - acc: 0.0202
Epoch 32/100
247/247 [==============================] - 0s 342us/step - loss: -49.3117 - acc: 0.0202
Epoch 33/100
247/247 [==============================] - 0s 343us/step - loss: -49.3117 - acc: 0.0202
Epo

247/247 [==============================] - 0s 344us/step - loss: -49.3117 - acc: 0.0202
Epoch 5/100
247/247 [==============================] - 0s 345us/step - loss: -49.3117 - acc: 0.0202
Epoch 6/100
247/247 [==============================] - 0s 338us/step - loss: -49.3117 - acc: 0.0202
Epoch 7/100
247/247 [==============================] - 0s 355us/step - loss: -49.3117 - acc: 0.0202
Epoch 8/100
247/247 [==============================] - 0s 341us/step - loss: -49.3117 - acc: 0.0202
Epoch 9/100
247/247 [==============================] - 0s 366us/step - loss: -49.3117 - acc: 0.0202
Epoch 10/100
247/247 [==============================] - 0s 349us/step - loss: -49.3117 - acc: 0.0202
Epoch 11/100
247/247 [==============================] - 0s 336us/step - loss: -49.3117 - acc: 0.0202
Epoch 12/100
247/247 [==============================] - 0s 348us/step - loss: -49.3117 - acc: 0.0202
Epoch 13/100
247/247 [==============================] - 0s 334us/step - loss: -49.3117 - acc: 0.0202
Epoch 14

247/247 [==============================] - 0s 349us/step - loss: -49.3117 - acc: 0.0202
Epoch 86/100
247/247 [==============================] - 0s 345us/step - loss: -49.3117 - acc: 0.0202
Epoch 87/100
247/247 [==============================] - 0s 349us/step - loss: -49.3117 - acc: 0.0202
Epoch 88/100
247/247 [==============================] - 0s 341us/step - loss: -49.3117 - acc: 0.0202
Epoch 89/100
247/247 [==============================] - 0s 344us/step - loss: -49.3117 - acc: 0.0202
Epoch 90/100
247/247 [==============================] - 0s 343us/step - loss: -49.3117 - acc: 0.0202
Epoch 91/100
247/247 [==============================] - 0s 341us/step - loss: -49.3117 - acc: 0.0202
Epoch 92/100
247/247 [==============================] - 0s 343us/step - loss: -49.3117 - acc: 0.0202
Epoch 93/100
247/247 [==============================] - 0s 342us/step - loss: -49.3117 - acc: 0.0202
Epoch 94/100
247/247 [==============================] - 0s 388us/step - loss: -49.3117 - acc: 0.0202
Epo

247/247 [==============================] - 0s 444us/step - loss: -49.3117 - acc: 0.0202
Epoch 66/100
247/247 [==============================] - 0s 369us/step - loss: -49.3117 - acc: 0.0202
Epoch 67/100
247/247 [==============================] - 0s 357us/step - loss: -49.3117 - acc: 0.0202
Epoch 68/100
247/247 [==============================] - 0s 541us/step - loss: -49.3117 - acc: 0.0202
Epoch 69/100
247/247 [==============================] - 0s 679us/step - loss: -49.3117 - acc: 0.0202
Epoch 70/100
247/247 [==============================] - 0s 369us/step - loss: -49.3117 - acc: 0.0202
Epoch 71/100
247/247 [==============================] - 0s 355us/step - loss: -49.3117 - acc: 0.0202
Epoch 72/100
247/247 [==============================] - 0s 349us/step - loss: -49.3117 - acc: 0.0202
Epoch 73/100
247/247 [==============================] - 0s 364us/step - loss: -49.3117 - acc: 0.0202
Epoch 74/100
247/247 [==============================] - 0s 717us/step - loss: -49.3117 - acc: 0.0202
Epo

In [29]:
type(score)
score= np.array(score)
print(np.mean(score[:,1]*100))

3.225806451612903


In [33]:
import shap
import numpy as np

# select a set of background examples to take an expectation over
background = X_train[np.random.choice(X_train.shape[0], 100, replace=False)]

# explain predictions of the model on four images
e = shap.DeepExplainer(model, background)
# ...or pass tensors directly
# e = shap.DeepExplainer((model.layers[0].input, model.layers[-1].output), background)
shap_values = e.shap_values(X_test[1:5])


In [34]:
shap_values

[array([[0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.]])]